In [ ]:
# Import the modules from skimage
from skimage import data, color
from matplotlib import pyplot as plt
import numpy as np
from matplotlib import image, patches
from skimage.filters import gaussian



In [ ]:
def show_image(image, title='Image', cmap_type='gray'):
    plt.subplots(figsize=(16, 12), sharex=True, sharey=True)
    plt.imshow(image, cmap = cmap_type)
    plt.title(title)
    plt.axis('off')
    plt.show

def plot_comparison(original, filtered, title_filtered):
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(16, 12), sharex=True, sharey=True)

    ax1.imshow(original, cmap=plt.cm.gray)
    ax1.set_title('original')
    ax1.axis('off')

    ax2.imshow(filtered, cmap=plt.cm.gray)
    ax2.set_title(title_filtered)
    ax2.axis('off')

def show_detected_face(result, detected, title='Face Image'):
    plt.imshow(result)
    img_desc = plt.gca()
    plt.set_cmap('gray')
    plt.title(title)
    plt.axis('off')

    for patch in detected:
        img_desc.add_patch(
            patches.Rectangle(
                (patch['c'], patch['r']),
                patch['width'],
                patch['height'],
                fill=False,color='r',linewidth=2
            )
        )
    plt.show

def getFace(d):
    #  X and Y starting points of the face rectangle
    x, y = d['r'], d['c']

    # The width and height of the face rectangle
    width, height = d['r'] + d['width'], d['c'] + d['height']

    # Extract the detected face
    face = image[x:width, y:height]
    return face

def mergeBlurryFace(original):
    # X and Y starting points of the face rectangle
    x, y = d['r'], d['c']
    
    # The width and height of the face rectangle
    width, height = d['r'] + d['width'], d['c'] + d['height']

    original[x:width, y:height] = gaussian(original[x:width, y:height], multichannel=True, sigma = 8)
    return original

def get_mask(image):
    mask = np.zeros(image.shape[:-1])
    mask[101:106, 0:240] = 1
    mask[152:154, 0:60] = 1
    mask[153:155, 60:100] = 1
    mask[154:156, 100:120] = 1
    mask[155:156, 120:140] = 1
    mask[212:217, 0:150] = 1
    mask[217:222, 150:256] = 1
    return mask

In [ ]:
# Import data
soaps_image = image.imread('data/chapter 4/toronjas.jpg')

# Import the filters module and sobel function
from skimage.filters import sobel

# Make the image grayscale
soaps_image_gray = color.rgb2gray(soaps_image)

# Apply edge detection filter
edge_sobel = sobel(soaps_image_gray)

# Show original and resulting image to compare
plot_comparison(soaps_image,edge_sobel,'Edges with Sobel')

In [ ]:
# Import the filters module and sobel function
from skimage.feature import canny

# Import data
grapefruit = image.imread('data/chapter 4/toronjas.jpg')

# Apply canny edge detector with a sigma of 1.8
edges_1_8 = canny(grapefruit, sigma=1.8)

# Apply canny edge detector with a sigma of 2.2
edges_2_2 = canny(grapefruit, sigma=2.2)

# Show resulting images
plot_comparison(edges_1_8, edges_2_2,'Sigma of 2.2')

In [ ]:
# Import the corner detector related functions and module
from skimage.feature import corner_harris, corner_peaks

# Import Image
building_image = image.imread('data/chapter 4/corners_building_top.jpg')

# Convert image from RGB-3 to grayscale
building_image_gray = color.rgb2gray(building_image)

# Apply the detector  to measure the possible corners
measure_image = corner_harris(building_image_gray)

# Find the peaks of the corners using the Harris detector
coords = corner_peaks(measure_image, min_distance=2)

# Show original and resulting image with corners detected
plot_comparison(building_image, coords,'Sigma of 2.2')

In [ ]:
# Import Cascade module
from skimage.feature import Cascade

# Import image
friends_image = image.imread('data/chapter 4/face_det_friends22.jpg')

# Load the trained file from data
trained_file = data.lbp_frontal_face_cascade_filename()

# Initialize the detector cascade
detector = Cascade(trained_file)

# Detect faces with scale factor to 1.2 and step ratio to 1
detected = detector.detect_multi_scale(img=friends_image,
                                       scale_factor=1.2,
                                       step_ratio=1,
                                       min_size=(10, 10),
                                       max_size=(200, 200))
# Show the detected faces
show_detected_face(friends_image, detected)

In [ ]:
from skimage.segmentation import slic
from skimage.color import label2rgb

profile_image = image.imread('data/chapter 4/face_det9.jpg')

# Obtain the segmentation with default 100 regions
segments = slic(profile_image, n_segments=100)

# Obtain segmented image using label2rgb
segmented_image = label2rgb(segments, profile_image, kind='avg')

# Detect the faces with multi scale method
detected = detector.detect_multi_scale(img=segmented_image, 
                                       scale_factor=1.2, 
                                       step_ratio=1, 
                                       min_size=(10, 10), max_size=(1000, 1000))

# Show the detected faces
show_detected_face(segmented_image, detected)

In [ ]:
group_image = image.imread('data/chapter 4/face_det25.jpg')

# Detect the faces
detected = detector.detect_multi_scale(img=group_image, 
                                       scale_factor=1.2, step_ratio=1, 
                                       min_size=(10,10), max_size=(100, 100))
# For each detected face
for d in detected:  
    # # Obtain the face rectangle from detected coordinates
    # face = getFace(d)
    
    # # Apply gaussian filter to extracted face
    # blurred_face = gaussian(face, multichannel=True, sigma = 8)
    
    # Merge this blurry face to our final image and show it
    resulting_image = mergeBlurryFace(group_image) 

show_image(resulting_image, "Blurred faces")

In [ ]:
# Import image 
damaged_image = image.imread('data/chapter 4/sally2.jpg')

# Import the necessary modules
from skimage.restoration import denoise_tv_chambolle, inpaint
from skimage import transform

# Transform the image so it's not rotated
upright_img = transform.rotate(damaged_image, 20)

# Remove noise from the image, using the chambolle method
upright_img_without_noise = denoise_tv_chambolle(upright_img,weight=0.1, multichannel=True)

# Reconstruct the image missing parts
mask = get_mask(upright_img)
result = inpaint.inpaint_biharmonic(upright_img_without_noise, mask, multichannel=True)

show_image(result)